In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def load_data(filepath):
    """
    Load and split data into training and test sets.
    """
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1].values  # Features
    y = data.iloc[:, -1].values-1   # Target
    return X, y

def build_and_train_mlp(x_train, x_test, y_train, y_test, hidden_layer_sizes, weight_decay, dropout_rate, learning_rate, epochs=100, batch_size=32):
    """
    Build, train, and evaluate an MLP with the specified parameters.
    """
    # Initialize the model
    model = Sequential()
    
    # Add hidden layers with dropout and L2 regularization
    for units in hidden_layer_sizes:
        model.add(Dense(units, activation='relu', kernel_regularizer=l2(weight_decay)))
        model.add(Dropout(dropout_rate))  # Dropout layer
    
    # Add output layer (assuming multi-class classification with softmax)
    model.add(Dense(len(np.unique(y_train)), activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), verbose=0)
    
    # Evaluate the model
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    y_pred = np.argmax(model.predict(x_test), axis=1)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return accuracy, f1

def main():
    # Load data
    X, y = load_data('C:/Users/Admin/Desktop/NN_assignment3/data/abalone.csv')
    
    # Normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Define parameter combinations
    param_combinations = [
        ((64, 32), 0.0001, 0.1, 0.01),  # Hidden layers (64, 32), weight decay 0.0001, dropout rate 0.1, learning rate 0.01
        ((64, 32), 0.001, 0.2, 0.01),   # Hidden layers (64, 32), weight decay 0.001, dropout rate 0.2, learning rate 0.01
        ((64, 32), 0.01, 0.3, 0.01),    # Hidden layers (64, 32), weight decay 0.01, dropout rate 0.3, learning rate 0.01
        ((64, 32), 0.0005, 0.1, 0.01),  # Additional combination for testing
        ((64, 32), 0.005, 0.2, 0.01)    # Additional combination for testing
    ]
    
    # Perform experiments
    results = []
    for hidden_layers, weight_decay, dropout_rate, learning_rate in param_combinations:
        # Split the data
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
        
        # Train and evaluate MLP with dropout
        accuracy, f1 = build_and_train_mlp(x_train, x_test, y_train, y_test, hidden_layer_sizes=hidden_layers, weight_decay=weight_decay, dropout_rate=dropout_rate, learning_rate=learning_rate)
        
        # Record results
        results.append({
            'Hidden Layers': hidden_layers,
            'Weight Decay': weight_decay,
            'Dropout Rate': dropout_rate,
            'Learning Rate': learning_rate,
            'Accuracy': accuracy,
            'F1 Score': f1
        })
        
        print(f"Hidden Layers: {hidden_layers}, Weight Decay: {weight_decay}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
    
    # Display final results
    for result in results:
        print(result)

if __name__ == '__main__':
    main()


53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step
Hidden Layers: (64, 32), Weight Decay: 0.0001, Dropout Rate: 0.1, Learning Rate: 0.01 - Accuracy: 0.6445, F1 Score: 0.6408
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step
Hidden Layers: (64, 32), Weight Decay: 0.001, Dropout Rate: 0.2, Learning Rate: 0.01 - Accuracy: 0.6355, F1 Score: 0.6172
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step
Hidden Layers: (64, 32), Weight Decay: 0.01, Dropout Rate: 0.3, Learning Rate: 0.01 - Accuracy: 0.6296, F1 Score: 0.6136
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Hidden Layers: (64, 32), Weight Decay: 0.0005, Dropout Rate: 0.1, Learning Rate: 0.01 - Accuracy: 0.6427, F1 Score: 0.6392
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Hidden Layers: (64, 32), Weight Decay: 0.005, Dropout Rate: 0.2, Learning Rate: 0.01 - Accuracy: 0.6517, F1 Score: 0.6422
{'Hidden Layers': (64, 32), 'Weight Decay': 0.0001, 'Dropout Rate': 0.1, 'Learning Rate': 0.01, 'Accuracy': 0.6445242166519165, 'F1 Score': np.float64(0.6407895257191938)}
{'Hidden Layers'